In [ ]:
# Plot and save file .csv delta F over F signal as a fraction -10-0-60 seconds.
# Don't forget to change to set the specific time to zero (recalibrate time)
# Save the processed data to a .csv file.

import pandas as pd
import numpy as np
import plotly.graph_objects as go
import os

# Define the file path for the CSV file
filename = "path/to/your/file.csv"  # Replace with your actual file path
output_dir = "directory/to/save/file.csv"  # Replace with your desired output directory
output_filename = 'name_of_the_file.csv'  # Replace with your desired output filename
output_filepath = os.path.join(output_dir, output_filename)

# Ensure the file exists and handle errors
try:
    data = pd.read_csv(filename)
except FileNotFoundError:
    raise FileNotFoundError(f"Error: File {filename} not found.")
except pd.errors.EmptyDataError:
    raise ValueError(f"Error: File {filename} is empty.")
except pd.errors.ParserError:
    raise ValueError(f"Error: File {filename} is not in a valid CSV format.")

# Identify cell columns (assuming they are named 'cell-07', 'cell-12', etc.)
cell_columns = [col for col in data.columns if col.lower().startswith('cell-')]

# Extract time and fluorescence data
time = data.iloc[:, 0]

# Set the specific time to zero (recalibrate time)
specific_time = 301.6  # seconds
recalibrated_time = time - specific_time

# Precompute index ranges for different windows
plot_start_idx = np.searchsorted(recalibrated_time, -10)
plot_end_idx = np.searchsorted(recalibrated_time, 60)
F0_start_idx = np.searchsorted(recalibrated_time, -10)
F0_end_idx = np.searchsorted(recalibrated_time, 0)

# Function to create a plot for signal data
def plot_signal(time, signal, title, y_label, cell, line_color='blue'):
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=time,
        y=signal,
        mode='lines',
        name=cell,
        line=dict(color=line_color, width=1)
    ))
    fig.update_layout(
        title=title,
        xaxis_title="Time (s)",
        yaxis_title=y_label,
        showlegend=False
    )
    fig.add_vline(x=0, line_dash="dash", line_color="red",
                  annotation_text="Zero point", annotation_position="top right")
    fig.show()

# Create a DataFrame to store the processed data for each cell
plot_info = pd.DataFrame()

# Loop through each cell trace
for cell in cell_columns:
    F_cell = data[cell]

    if F_cell.isnull().all():
        print(f"Skipping {cell} as all values are NaN")
        continue

    # Calculate F0 for the specific time window (-10 to 0 seconds)
    F0 = np.mean(F_cell[F0_start_idx:F0_end_idx])
    if F0 == 0:
        print(f"Skipping {cell} as F0 is zero, leading to division by zero.")
        continue

    # Calculate delta F/F (%)
    delta_FF = (F_cell - F0) / np.abs(F0)

    # Select the time and signal windows for plotting (-10 to 60 seconds)
    time_window = recalibrated_time[plot_start_idx:plot_end_idx]
    raw_signal_window = F_cell[plot_start_idx:plot_end_idx]
    dFF_signal_window = delta_FF[plot_start_idx:plot_end_idx]

    # Append the dF/F signal to the plot information DataFrame
    cell_data = pd.DataFrame({f"{cell}": dFF_signal_window})
    plot_info = pd.concat([plot_info, cell_data], axis=1)

    # Plot the original fluorescence signal
    plot_signal(
        time_window, raw_signal_window,
        title=f"Original Fluorescence Signal - {cell}",
        y_label="Fluorescence Intensity",
        cell=cell,
        line_color='green'
    )

    # Plot the delta F/F signal
    plot_signal(
        time_window, dFF_signal_window,
        title=f"Delta F/F Signal - {cell}",
        y_label="dF/F (fraction)",
        cell=cell,
        line_color='blue'
    )

# Insert the recalibrated time column at the start of plot_info DataFrame
plot_info.insert(0, 'time', time_window)

# Save the plot information DataFrame to a .csv file
plot_info.to_csv(output_filepath, index=False)
print(f"Processed data saved to {output_filepath}")
